In [ ]:
# 넷플릭스 영화 추천 시스템 구현
import pandas as pd
import numpy as np
from math import sqrt

In [ ]:
#데이터 읽어오기
# 1205 현재 위치 -> 2개 csv 존재
movies = pd.read_csv("./movies.csv")
ratings = pd.read_csv("./ratings.csv")

print(ratings)

         userId  movieId  rating   timestamp
0             1        2     3.5  1112486027
1             1       29     3.5  1112484676
2             1       32     3.5  1112484819
3             1       47     3.5  1112484727
4             1       50     3.5  1112484580
...         ...      ...     ...         ...
1048570    7120      168     5.0  1175543061
1048571    7120      253     4.0  1175542225
1048572    7120      260     5.0  1175542035
1048573    7120      261     4.0  1175543376
1048574    7120      266     3.5  1175542454

[1048575 rows x 4 columns]


In [ ]:
# 아이템 기반 협업 필터링 구현
data = pd.merge(ratings, movies, on="movieId")
column = ['userId', 'movieId', 'rating', 'title', 'genres']
data = data[column]

moviedata = data.pivot_table(index="movieId", columns='userId')['rating']

# NaN 값을 -1 로 변경
moviedata.fillna(-1, inplace=True)


def sim_distance(data, n1, n2):
    # kdd 유사도 함수
    sum = 0
    # 두 사용자가 모두 본 영화를 기준
    for i in data.loc[n1, data.loc[n1, :] >= 0].index:
        if data.loc[n2, i] >= 0:
            # 누적합
            sum += pow(data.loc[n1, i]-data.loc[n2, i], 2)
    return sqrt(1/(sum+1))  # 유사도 형식으로 출력

In [ ]:
def top_match(data, name, rank = 5, simf = sim_distance):
  # 나와 유사도가 높은 유저 매칭
  simList = []
  for i in data.index[-10:]:
    if name !=i :
      simList.append((simf(data, name, i), i))
    simList.sort()
    simList.reverse()
    return simList[:rank]

In [ ]:
def recommendation(data, person, simf=sim_distance):
    # 추천 시스템 함수
    result_top = top_match(data, person, len(data))
    score_dic = {}
    sim_dic = {}
    my_list = []
    for sim, name in result_top:
        if sim < 0:
            continue
        for movie in data.loc[person, data.loc[person, :] < 0].index:
            sim_sum = 0
            if data.loc[name, movie] >= 0:
                sim_sum += data.loc[name, movie]

                score_dic.setdefault(movie, 0)
                score_dic[movie] += sim_sum

                sim_dic.setdefault(movie, 0)
                sim_dic[movie] += sim

    for key in score_dic:
        my_list.append((score_dic[key]/sim_dic[key], key))
    my_list.sort()
    my_list.reverse()
    return my_list

In [ ]:
# 20번 user가 안본 영화중에서 추천 점수가 가장 높은 순으로 예상 평점과 영화 제목을 추천(15개까지)
movie_list = []
for rate, m_id in recommendation(moviedata, 20):
    movie_list.append((rate, movies.loc[movies['movieId'] == m_id,
                                        'title'].values[0]))
    if len(movie_list) == 15:
        break

print(movie_list[:15])

[(3.5, 'Born Yesterday (1993)')]
